In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers,callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler,Callback,EarlyStopping

In [ ]:
print("GPU Available:", tf.test.is_gpu_available())

# Data Loading

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df

In [ ]:
df.describe()

In [ ]:
y = df.iloc[:,0].copy()
X = df.iloc[:,1:] 
X = X.values.reshape(df.shape[0],28,28,1) / 255 #normalizing the data


In [ ]:
# Create a figure for visualization
plt.figure(figsize=(10, 15))

for i in range(10):
    plt.subplot(5, 5, i+1)
    
    plt.grid(False)
    
    plt.imshow(X[i], cmap='gray')
    
    actual_label = y.iloc[i]
    
    plt.xlabel(actual_label)
plt.show()

In [ ]:
sns.barplot(x = y.unique(), y = y.value_counts())
plt.title('Counts Of All categorical Labels')
plt.show()

In [ ]:
y = to_categorical(y, num_classes=10)
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size =0.3, random_state = 42)

X_train = tf.constant(X_train)
X_test = tf.constant(X_test)
y_train = tf.constant(y_train)
y_test = tf.constant(y_test)

In [ ]:
early_stopping = EarlyStopping(patience=5, monitor='accuracy', restore_best_weights = True)

class myCallbacks(Callback):
    def on_epochs_end(self,epoch,logs={}):
        target_accuracy = 0.995
        target_loss = 0.015
        
        if (logs.get('accuracy') >= target_accuracy) or (logs.get('loss') <= target_loss):
            print('Model Has Reached Accuracy of 99.5% and loss has been decreased to 0.015')
            print('Epoch {}: loss = {:.4f}, accuracy = {:.4f}'.format(epoch, logs['loss'], logs['accuracy']))
            self.model.stop_training = True
callthis = myCallbacks()

In [ ]:
model = Sequential(
    [
        layers.InputLayer(input_shape=[28,28,1]),
        
        #stating hidden layers
        layers.Conv2D(28, kernel_size=(2,2),activation='relu'),
        layers.MaxPool2D(pool_size=(2,2)),
        
        layers.Conv2D(56,kernel_size=(2,2), activation='relu',padding='same'),
        layers.MaxPool2D(pool_size=(2,2)),
        
        layers.Flatten(),
        layers.Dense(20, activation='leaky_relu'),
        layers.Dropout(0.2),
        layers.Dense(40, activation='tanh'),
        
        layers.Dense(10, activation='softmax')
    ]
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
model.summary()

In [ ]:
cnn_ = model.fit(X_train,y_train, epochs=50, batch_size=10, callbacks=[early_stopping,callthis])
model.evaluate(X_test,y_test)

In [ ]:
model_history = cnn_.history

plt.figure(figsize=(12,7))
plt.grid(True)
plt.plot(model_history['loss'])
plt.plot(model_history['accuracy'])

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

y_pred_probs = model.predict(X_test)

y_test_bin = label_binarize(y_test, classes=np.arange(10))

fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 10

plt.figure(figsize=(10, 6))

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_pred_probs.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'blue', 'purple', 'pink', 'brown', 'gray'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (class {0}) (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
         ''.format(roc_auc["micro"]), color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiple Class ROC Curves')
plt.legend(loc="lower right")
plt.show()